# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [ ]:
# imports
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr


In [ ]:
# setup - connect to OpenAI and Ollama
load_dotenv(override=True)
api_key = os.getenv("OPENAI_API_KEY")

openai = OpenAI()
ollama = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")

MODELS = {
    "GPT-4o-mini": ("gpt-4o-mini", openai),
    "Llama 3.2 (Ollama)": ("llama3.2", ollama),
}


In [ ]:
# expert system prompt for technical Q&A
SYSTEM_PROMPT = """You are an expert technical assistant who explains code, APIs, and programming concepts clearly.
You help developers understand complex technical questions. Use markdown for code blocks and structure.
Be concise but thorough. When explaining code, break it down step by step."""


In [ ]:
# Bonus: simple tool - evaluate safe math expressions
def evaluate_expression(expression: str) -> str:
    allowed = set("0123456789+-*/(). ")
    if not all(c in allowed for c in expression):
        return "Error: Only numbers and + - * / ( ) allowed"
    try:
        return f"Result: {eval(expression)}"
    except Exception as e:
        return f"Error: {e}"

eval_tool = {"type": "function", "function": {"name": "evaluate_expression", "description": "Evaluate a math expression.", "parameters": {"type": "object", "properties": {"expression": {"type": "string"}}, "required": ["expression"], "additionalProperties": False}}}
tools = [eval_tool]


In [ ]:
def handle_tool_calls(message):
    responses = []
    for tool_call in message.tool_calls:
        if tool_call.function.name == "evaluate_expression":
            args = json.loads(tool_call.function.arguments)
            responses.append({"role": "tool", "content": evaluate_expression(args.get("expression", "")), "tool_call_id": tool_call.id})
    return responses


In [ ]:
def chat(message, history, model_choice):
    model_name, client = MODELS[model_choice]
    history = [{"role": h["role"], "content": h["content"]} for h in history]
    messages = [{"role": "system", "content": SYSTEM_PROMPT}] + history + [{"role": "user", "content": message}]
    stream = client.chat.completions.create(model=model_name, messages=messages, stream=True)
    full_response = ""
    for chunk in stream:
        if chunk.choices and chunk.choices[0].delta.content:
            full_response += chunk.choices[0].delta.content or ""
            yield full_response


In [ ]:
model_dropdown = gr.Dropdown(choices=list(MODELS.keys()), value="GPT-4o-mini", label="Model")
gr.ChatInterface(fn=chat, type="messages", additional_inputs=[model_dropdown]).launch()
